# Day 5: If You Give A Seed A Fertilizer

[*Advent of Code 2023 day 5*](https://adventofcode.com/2023/day/5) and [*solution megathread*](https://redd.it/18b4b0r)

[![nbviewer](https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg)](https://nbviewer.jupyter.org/github/UncleCJ/advent-of-code/blob/cj/2023/05/code.ipynb) [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/UncleCJ/advent-of-code/cj?filepath=2023%2F05%2Fcode.ipynb)

In [1]:
from IPython.display import HTML
import sys
sys.path.append('../../')


# %load_ext nb_mypy
# %nb_mypy On

In [2]:
import common


downloaded = common.refresh()
%store downloaded >downloaded

# %load_ext pycodestyle_magic
# %pycodestyle_on

Writing 'downloaded' (dict) to file 'downloaded'.


In [3]:
from IPython.display import HTML

HTML(downloaded['part1'])

In [4]:
example_input = '''seeds: 79 14 55 13

seed-to-soil map:
50 98 2
52 50 48

soil-to-fertilizer map:
0 15 37
37 52 2
39 0 15

fertilizer-to-water map:
49 53 8
0 11 42
42 0 7
57 7 4

water-to-light map:
88 18 7
18 25 70

light-to-temperature map:
45 77 23
81 45 19
68 64 13

temperature-to-humidity map:
0 69 1
1 0 69

humidity-to-location map:
60 56 37
56 93 4'''

In [5]:
from typing import NamedTuple

class MapRange(NamedTuple):
    y_start: int
    x_start: int
    count: int

    def x_end(self) -> int:
        return self.x_start + self.count - 1

In [6]:
from typing import List, Tuple
import re
from operator import attrgetter

def parse_almanac_map(lines: List[str]) -> Tuple[
        str,
        str,
        Tuple[MapRange, ...]
        ]:
    if m := re.match(
            r'^(?P<x_cat>[a-z]+)-to-(?P<y_cat>[a-z]+) map:$',
            lines[0]
            ):
        gd = m.groupdict()
        x_cat, y_cat = gd['x_cat'], gd['y_cat'] 
    ranges = tuple(sorted((
        MapRange(*map(int, line.split()))
        for line
        in lines[1:]
        ),
        key=attrgetter('x_start')))
    return x_cat, y_cat, ranges

parse_almanac_map('''water-to-light map:
88 18 7
18 25 70'''.splitlines())

('water',
 'light',
 (MapRange(y_start=88, x_start=18, count=7),
  MapRange(y_start=18, x_start=25, count=70)))

In [7]:
from typing import Iterable
from itertools import chain

def parse_input(lines: List[str]) -> Tuple[
        Tuple[int, ...],
        Tuple[str, ...],
        Tuple[Tuple[MapRange, ...], ...]
        ]:
    def split_blank_lines(lines: List[str]) -> Iterable[Iterable[str]]:
        blanklines = [
            row
            for row, line
            in enumerate(lines)
            if line == ''
            ]
        for start_row, stop_row in zip(
                map(
                    lambda r: r + 1,
                    [-1] + blanklines
                    ),
                blanklines + [len(lines)]):
            yield lines[start_row:stop_row]

    seeds = tuple(map(int, re.findall(r'\d+', lines[0])))
    almanac_maps = {
        (source_cat, dest_cat): ranges
        for source_cat, dest_cat, ranges
        in map(
            parse_almanac_map,
            split_blank_lines(lines[2:])
            )
    }
    categories = tuple(set(chain(*almanac_maps.keys())))
    return seeds, categories, tuple(almanac_maps.values())

seeds, categories, maps = parse_input(example_input.splitlines())
print(f'{seeds=},\n {categories=},\n {maps=}')

seeds=(79, 14, 55, 13),
 categories=('water', 'light', 'humidity', 'temperature', 'location', 'fertilizer', 'seed', 'soil'),
 maps=((MapRange(y_start=52, x_start=50, count=48), MapRange(y_start=50, x_start=98, count=2)), (MapRange(y_start=39, x_start=0, count=15), MapRange(y_start=0, x_start=15, count=37), MapRange(y_start=37, x_start=52, count=2)), (MapRange(y_start=42, x_start=0, count=7), MapRange(y_start=57, x_start=7, count=4), MapRange(y_start=0, x_start=11, count=42), MapRange(y_start=49, x_start=53, count=8)), (MapRange(y_start=88, x_start=18, count=7), MapRange(y_start=18, x_start=25, count=70)), (MapRange(y_start=81, x_start=45, count=19), MapRange(y_start=68, x_start=64, count=13), MapRange(y_start=45, x_start=77, count=23)), (MapRange(y_start=1, x_start=0, count=69), MapRange(y_start=0, x_start=69, count=1)), (MapRange(y_start=60, x_start=56, count=37), MapRange(y_start=56, x_start=93, count=4)))


In [8]:
def apply_almanac_map(x: int, almanac_map: Tuple[MapRange, ...]) -> int:
    for map_range in almanac_map:
        if map_range.x_start <= x <= map_range.x_end():
            return map_range.y_start + (x - map_range.x_start)
    else:
        return x

apply_almanac_map(79, maps[0])

81

In [9]:
from typing import Dict
from itertools import accumulate

def apply_almanac_maps(
        categories: Tuple[str, ...],
        maps: Tuple[MapRange, ...],
        x: int
        ) -> Dict[str, int]:
    return {
        category: y
        for category, y
        in zip(
            categories,
            accumulate(maps, func=apply_almanac_map, initial=x)
            )
        }

apply_almanac_maps(categories, maps, 79)

{'water': 79,
 'light': 81,
 'humidity': 81,
 'temperature': 81,
 'location': 74,
 'fertilizer': 78,
 'seed': 78,
 'soil': 82}

In [10]:
def str_list(ls: List) -> str:
    return '[\n\t' + ',\n\t'.join(str(l) for l in ls) + '\n]'

print(str_list(maps))

[
	(MapRange(y_start=52, x_start=50, count=48), MapRange(y_start=50, x_start=98, count=2)),
	(MapRange(y_start=39, x_start=0, count=15), MapRange(y_start=0, x_start=15, count=37), MapRange(y_start=37, x_start=52, count=2)),
	(MapRange(y_start=42, x_start=0, count=7), MapRange(y_start=57, x_start=7, count=4), MapRange(y_start=0, x_start=11, count=42), MapRange(y_start=49, x_start=53, count=8)),
	(MapRange(y_start=88, x_start=18, count=7), MapRange(y_start=18, x_start=25, count=70)),
	(MapRange(y_start=81, x_start=45, count=19), MapRange(y_start=68, x_start=64, count=13), MapRange(y_start=45, x_start=77, count=23)),
	(MapRange(y_start=1, x_start=0, count=69), MapRange(y_start=0, x_start=69, count=1)),
	(MapRange(y_start=60, x_start=56, count=37), MapRange(y_start=56, x_start=93, count=4))
]


In [11]:
from functools import partial

seeds, categories, maps = parse_input(example_input.splitlines())
# seeds, categories, maps = parse_input(downloaded['input'].splitlines())
results = list(map(
    partial(apply_almanac_maps, categories, maps), 
    seeds
    ))
print(str_list(results))
print(min(r['location'] for r in results))

[
	{'water': 79, 'light': 81, 'humidity': 81, 'temperature': 81, 'location': 74, 'fertilizer': 78, 'seed': 78, 'soil': 82},
	{'water': 14, 'light': 14, 'humidity': 53, 'temperature': 49, 'location': 42, 'fertilizer': 42, 'seed': 43, 'soil': 43},
	{'water': 55, 'light': 57, 'humidity': 57, 'temperature': 53, 'location': 46, 'fertilizer': 82, 'seed': 82, 'soil': 86},
	{'water': 13, 'light': 13, 'humidity': 52, 'temperature': 41, 'location': 34, 'fertilizer': 34, 'seed': 35, 'soil': 35}
]
34


In [12]:
HTML(downloaded['part2'])

In [13]:
print(seeds)

(79, 14, 55, 13)


In [14]:
seedranges = [(seeds[2*i], seeds[2*i+1]) for i in range(len(seeds)//2)]
print(str_list(seedranges))

[
	(79, 14),
	(55, 13)
]


In [15]:
from typing import Optional


def apply_maprange_to_range(current_range: MapRange, x_start: int, x_count: int) -> MapRange:
    x_end = x_start + x_count - 1
    if x_start < current_range.x_start:
        if x_end <= current_range.x_end():
            return MapRange(
                current_range.y_start,
                current_range.x_start,
                x_end - current_range.x_start + 1
                )
        else:  # x_end >= current_range.x_end()
            return current_range
    else:  # x_start >= current_range.x_start
        if x_end <= current_range.x_end():
            return MapRange(
                (x_start - current_range.x_start) + current_range.y_start,
                x_start,
                x_count
                )
        else:  # x_end >= current_range.x_end()
            return MapRange(
                (x_start - current_range.x_start) + current_range.y_start,
                x_start,
                current_range.count - (x_start - current_range.x_start)
                )
   
	# Tuple(MapRange(y_start=50, x_start=98, count=2), MapRange(y_start=52, x_start=50, count=48))

apply_maprange_to_range(
    MapRange(50, 75, 10),
    70,
    18)

MapRange(y_start=50, x_start=75, count=10)

In [16]:
from typing import Iterator

def apply_map_to_range(
        almanac_map: Tuple[MapRange, ...],
        x_start: int,
        x_count: int) -> Iterator[MapRange]:
    # print(f"{almanac_map=}, {x_start=}, {x_count=}")
    x_end = x_start + x_count - 1
    acc_start = x_start
    # Assume the maps are sorted but may or may not overlap with x_end
    for map_range in almanac_map:
        # print(f"{map_range=}, {acc_start=}, {x_end=}")
        if x_start <= map_range.x_end() and x_end >= map_range.x_start:
            if acc_start < map_range.x_start:
                yield MapRange(acc_start, acc_start, map_range.x_start - acc_start)
            yield apply_maprange_to_range(map_range, x_start, x_count)
            if map_range.x_end() >= x_end:
                break
            if x_end > map_range.x_end():
                acc_start = map_range.x_end() + 1
    else:
        yield MapRange(acc_start, acc_start, x_end - acc_start + 1)

print(maps[2])
print(seedranges)
list(apply_map_to_range(maps[2], *seedranges[1]))

(MapRange(y_start=42, x_start=0, count=7), MapRange(y_start=57, x_start=7, count=4), MapRange(y_start=0, x_start=11, count=42), MapRange(y_start=49, x_start=53, count=8))
[(79, 14), (55, 13)]


[MapRange(y_start=51, x_start=55, count=6),
 MapRange(y_start=61, x_start=61, count=7)]

In [17]:
def apply_map_to_map(
        almanac_map: Tuple[MapRange, ...],
        x_map: Tuple[MapRange, ...]) -> Tuple[MapRange, ...]:
    return tuple(
        sorted(
            chain(*(
                apply_map_to_range(almanac_map, y_start, count)
                for y_start, _, count
                in x_map
                )),
            key=attrgetter('x_start')
            )
        )

apply_map_to_map(maps[2], tuple([MapRange(seedranges[0][0], seedranges[0][0], seedranges[0][1]), MapRange(seedranges[1][0], seedranges[1][0], seedranges[1][1])]))

(MapRange(y_start=51, x_start=55, count=6),
 MapRange(y_start=61, x_start=61, count=7),
 MapRange(y_start=79, x_start=79, count=14))

In [18]:
def apply_maps_to_range(
        categories: Tuple[str, ...],
        maps: Tuple[MapRange, ...],
        x_start: int,
        x_count: int
        ):
    return {
        category: y
        for category, y
        in zip(
            categories,
            accumulate(maps, func=apply_map_to_map, initial=(MapRange(x_start, x_start, x_count),))
            )
        }

apply_maps_to_range(categories, maps, seedranges[0][0], seedranges[0][1])

{'water': (MapRange(y_start=79, x_start=79, count=14),),
 'light': (MapRange(y_start=50, x_start=50, count=2),
  MapRange(y_start=52, x_start=52, count=27),
  MapRange(y_start=79, x_start=79, count=14),
  MapRange(y_start=93, x_start=93, count=7)),
 'humidity': (MapRange(y_start=0, x_start=0, count=37),
  MapRange(y_start=37, x_start=37, count=2),
  MapRange(y_start=39, x_start=39, count=11),
  MapRange(y_start=50, x_start=50, count=2),
  MapRange(y_start=52, x_start=52, count=2)),
 'temperature': (MapRange(y_start=0, x_start=0, count=37),
  MapRange(y_start=37, x_start=37, count=2),
  MapRange(y_start=39, x_start=39, count=3),
  MapRange(y_start=42, x_start=42, count=7),
  MapRange(y_start=49, x_start=49, count=1),
  MapRange(y_start=50, x_start=50, count=2),
  MapRange(y_start=52, x_start=52, count=2),
  MapRange(y_start=54, x_start=54, count=3),
  MapRange(y_start=57, x_start=57, count=4)),
 'location': (MapRange(y_start=18, x_start=18, count=19),
  MapRange(y_start=37, x_start=37, 